In [ ]:
import gurobipy as gp
from gurobipy import GRB

def solve_freeloading_unoptimized():
    """
    Implementa o modelo 'position-free' (freeloading) exatamente como descrito
    no documento, sem nenhuma otimização de performance.

    Este modelo é intencionalmente lento e serve para fins de comparação.
    """
    try:
        # --- 1. Dados do Modelo ---
        L, W, H = 12, 8, 8
        boxes_dims = [
            (6, 3, 2), (6, 3, 2), (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3), (6, 4, 3),
            (8, 3, 2), (8, 3, 2), (8, 3, 2), (4, 3, 2), (4, 3, 2), (4, 4, 3), (4, 4, 3), (4, 4, 3)
        ]
        m = len(boxes_dims)
        values = [l*w*h for l,w,h in boxes_dims]

        # --- 2. Parâmetros da Formulação Original ---

        # M grande para as restrições "big-M"
        M = L + W + H + 1000 # Precisa ser grande o suficiente para o espaço com Xo

        # Coordenadas de início do contêiner para separação espacial [cite: 77]
        # Esta é a principal causa da ineficiência
        Xo, Yo, Zo = 1000, 1000, 1000

        # --- 3. Criação do Modelo ---
        model = gp.Model("FreeloadingUnoptimized")
        model.setParam('OutputFlag', 1) # Habilita o log para ver o quão lento é

        # --- 4. Variáveis de Decisão [cite: 78-83] ---
        p = model.addVars(m, vtype=GRB.BINARY, name="p")
        x = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="x")
        y = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="y")
        z = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0, name="z")
        pairs = [(i, j) for i in range(m) for j in range(m) if i < j]
        a = model.addVars(pairs, vtype=GRB.BINARY, name="a")
        b = model.addVars(pairs, vtype=GRB.BINARY, name="b")
        c = model.addVars(pairs, vtype=GRB.BINARY, name="c")
        d = model.addVars(pairs, vtype=GRB.BINARY, name="d")
        e = model.addVars(pairs, vtype=GRB.BINARY, name="e")
        f = model.addVars(pairs, vtype=GRB.BINARY, name="f")

        # --- 5. Função Objetivo [cite: 101] ---
        model.setObjective(gp.quicksum(values[i] * p[i] for i in range(m)), GRB.MAXIMIZE)

        # --- 6. Restrições (Formulação Original) ---

        # A. Restrições de Não Sobreposição Ineficientes
        for i, j in pairs:
            li, wi, hi = boxes_dims[i]
            lj, wj, hj = boxes_dims[j]

            model.addConstr(x[i] + li <= x[j] + M * (1 - a[i, j]))
            model.addConstr(x[j] + lj <= x[i] + M * (1 - b[i, j]))
            model.addConstr(y[i] + wi <= y[j] + M * (1 - c[i, j]))
            model.addConstr(y[j] + wj <= y[i] + M * (1 - d[i, j]))
            model.addConstr(z[i] + hi <= z[j] + M * (1 - e[i, j]))
            model.addConstr(z[j] + hj <= z[i] + M * (1 - f[i, j]))

            # Esta restrição é aplicada a TODAS as caixas, empacotadas ou não [cite: 120-122].
            # O solver é forçado a encontrar um layout válido para as caixas não utilizadas.
            model.addConstr(a[i, j] + b[i, j] + c[i, j] + d[i, j] + e[i, j] + f[i, j] >= 1, f"inefficient_non_overlap_{i}_{j}")

        # B. Restrições de Posicionamento com Separação Espacial [cite: 125-142]
        for i in range(m):
            li, wi, hi = boxes_dims[i]

            # Envia a caixa para a região do contêiner (distante) se p[i]=1
            model.addConstr(x[i] >= Xo * p[i], f"place_x_{i}")
            model.addConstr(y[i] >= Yo * p[i], f"place_y_{i}")
            model.addConstr(z[i] >= Zo * p[i], f"place_z_{i}")

            # Define os limites do espaço total (incluindo a "sala de descarte")
            # Essas restrições de limite superior são fracas e numericamente ruins
            model.addConstr(x[i] + li <= Xo + L, f"bound_x_high_{i}")
            model.addConstr(y[i] + wi <= Yo + W, f"bound_y_high_{i}")
            model.addConstr(z[i] + hi <= Zo + H, f"bound_z_high_{i}")

        # --- 7. Otimização ---
        print("--- Iniciando otimização com o modelo original (não otimizado)... ---")
        print("AVISO: Esta execução pode ser extremamente lenta.")
        model.optimize()

        # --- 8. Resultados ---
        if model.SolCount > 0:
            print("\n" + "="*40)
            print("      SOLUÇÃO ENCONTRADA")
            print("="*40)
            print(f"Volume Total Empacotado: {model.ObjVal}")
            packed_boxes = [i for i in range(m) if p[i].X > 0.5]
            print(f"Número de caixas empacotadas: {len(packed_boxes)}")
            for i in packed_boxes:
                # Ajusta as coordenadas para serem relativas ao contêiner
                rel_x = x[i].X - Xo
                rel_y = y[i].X - Yo
                rel_z = z[i].X - Zo
                print(f"  - Caixa {i+1}: Coordenadas ({rel_x:.2f}, {rel_y:.2f}, {rel_z:.2f})")
        else:
            print("\nNenhuma solução encontrada.")

    except gp.GurobiError as e:
        print(f"Erro do Gurobi: {e}")

# Executa o modelo não otimizado
solve_freeloading_unoptimized()

Set parameter Username
Set parameter LicenseID to value 2707346
Academic license - for non-commercial use only - expires 2026-09-11
Set parameter OutputFlag to value 1
--- Iniciando otimização com o modelo original (não otimizado)... ---
AVISO: Esta execução pode ser extremamente lenta.
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 825 rows, 690 columns and 2655 nonzeros
Model fingerprint: 0xde8fc5f2
Variable types: 45 continuous, 645 integer (645 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 45 rows and 0 columns
Presolve time: 0.00s
Presolved: 780 rows, 690 columns, 2610 nonzeros
Variable types: 45 continuous, 645 integer (645 binary)

Root relax